# Plotting PyPSA networks along the workflow

### Jupyter scripts need to be improved
#### It seems that hv.plot is not working with `pypsa-africa`. Needs a fix.

In this script, the ouputs of the PyPSA networks calculated in the Data Workflow are computed.
In particular, the considered outputs are of the following scripts:
- osm_data_cleaning
- base_network
- add_electricity
- simplify_network
- cluster_network

## Specify file paths of the files related to the scripts

In [ ]:
# Specify the current directory
import sys
sys.path.append('../')  # to import helpers

from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")

In [ ]:
# file paths of osm_data_cleaning outputs
import os
substations_ODC_path = os.getcwd() + '/data/clean/africa_all_substations.geojson'
lines_ODC_path = os.getcwd() + '/data/clean/africa_all_lines.geojson'
# base_network file path
network_path_b = os.getcwd() + '/networks/base.nc'
# add_electricity file path
network_path_a = os.getcwd() + '/networks/elec.nc'
# simplify_network file path
network_path_s = os.getcwd() + '/networks/elec_s.nc'
# cluster_network file path
network_path_c = os.getcwd() + '/networks/elec_s_10.nc'

## Load python packages

In [ ]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib as plt

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared
from shapely.wkt import loads

logger = logging.getLogger(__name__)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 70)

## Plot network of osm_data_cleaning

In [ ]:
# plot output osm_data_cleaning
df_substations_osm_clean = gpd.read_file(substations_ODC_path)
df_lines_osm_clean = gpd.read_file(lines_ODC_path)

import hvplot.pandas

df_substations_osm_clean.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    color = 'orange'  
) * df_lines_osm_clean.hvplot(
    geo=True,
    alpha=0.4,
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## Plot network of base_network

In [ ]:
n_b = pypsa.Network(network_path_b)

lines_b = n_b.lines
lines_b['geometry'] = lines_b['geometry'].apply(loads)
lines_b = gpd.GeoDataFrame(lines_b, crs='epsg:4326')

buses_b = n_b.buses
buses_b["geometry"] = gpd.points_from_xy(buses_b.lon, buses_b.lat)
buses_b = gpd.GeoDataFrame(buses_b, crs='epsg:4326')

buses_b.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines_b.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## Plot network of add_electricity

In [ ]:
n_a = pypsa.Network(network_path_a)

lines_a = n_a.lines
lines_a['geometry'] = lines_a['geometry'].apply(loads)
lines_a = gpd.GeoDataFrame(lines_a, crs='epsg:4326')

buses_a = n_a.buses
buses_a["geometry"] = gpd.points_from_xy(buses_a.lon, buses_a.lat)
buses_a = gpd.GeoDataFrame(buses_a, crs='epsg:4326')

buses_a.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines_a.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## Plot network of simplify_network

In [ ]:
n_s = pypsa.Network(network_path_s)

lines_s = n_s.lines
lines_s['geometry'] = lines_s['geometry'].apply(loads)
lines_s = gpd.GeoDataFrame(lines_s, crs='epsg:4326')

buses_s = n_s.buses
buses_s["geometry"] = gpd.points_from_xy(buses_s.lon, buses_s.lat)
buses_s = gpd.GeoDataFrame(buses_s, crs='epsg:4326')

buses_s.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines_s.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)

## Plot network of cluster_network

In [ ]:
n_c = pypsa.Network(network_path_c)

buses_c = n_c.buses
buses_c["geometry"] = gpd.points_from_xy(buses_c.x, buses_c.y)
buses_c = gpd.GeoDataFrame(buses_c, crs='epsg:4326')

lines_c = n_c.lines
lines_c['geometry'] = lines_c.apply(lambda x: LineString([buses_c.loc[x["bus0"], "geometry"], buses_c.loc[x["bus1"], "geometry"]]), axis=1)
lines_c = gpd.GeoDataFrame(lines_c, crs='epsg:4326')

buses_c.hvplot(
    geo=True,
    size = 10,  # buses["tag_area"]**(0.5)/10,
    frame_height=750,
    alpha=0.4,
    tiles='CartoLight',
    hover_cols=['bus_id'],
    color = 'orange'  
) * lines_c.hvplot(
    geo=True,
    alpha=0.4,
    hover_cols=['line_id']
).opts(
    active_tools=['pan', 'wheel_zoom']
)